In [4]:
import os
import datetime
import quandl

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import math
from scipy import stats
import itertools
import pandas_datareader.data as web
from functools import reduce
import random


In [5]:
# File Constants
DATA_FILE = "historical_options_IBM.csv"
TICKER_FILE = "tickersf.txt"
EOD_DATA_FILE = "eod_prices.csv"

In [ ]:
def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    root_data_dir = os.path.join(os.environ["HOME"], "Downloads")
    data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink
    
    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.mkdir(table_dir)

    if date_override is None:
        my_date = datetime.datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    dl = quandl.export_table(
        table_path, filename=data_file, api_key="TDynpHNbX5ucpahcz_85"
, **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(
                data_file, data_symlink,
            )
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"


def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
    )

In [7]:
prices_eod_primary = fetch_quandl_table('QUOTEMEDIA/PRICES', avoid_download=True)

Skipping any possible download of QUOTEMEDIA/PRICES


In [8]:
def merge_prices_ratio(df, eod_prices):
    eod_prices = eod_prices.sort_values(["date", "ticker",])
    df = df.sort_values(["per_end_date", "ticker"])
    df["per_end_date"] = pd.to_datetime(df["per_end_date"])
    eod_prices["date"] = pd.to_datetime(eod_prices["date"])
    merged_df = pd.merge_asof(
        df,
        eod_prices[["date", "ticker", "adj_close"]],
        left_on="per_end_date",    # the target date
        right_on="date",           # the trading date
        by="ticker",               # match within each ticker
        direction="backward"       # use last trading day on or before per_end_date
    )
    return merged_df

In [ ]:
ibm = pd.read_csv(DATA_FILE, header=None)
ibm.reset_index(inplace=True)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,date,implied_volatility,delta,gamma,theta,vega,rho
1,IBM171117C00075000,IBM,2017-11-17,75.00,call,38.35,72.28,70.45,4,74.10,5,0,0,2017-11-15,4.20571,0.98982,0.00059,-0.31226,0.00295,0.00402
2,IBM171117P00075000,IBM,2017-11-17,75.00,put,0.00,0.01,0.00,0,0.03,50,0,0,2017-11-15,3.06259,-0.00102,0.00010,-0.02852,0.00037,-0.00001
3,IBM171117C00080000,IBM,2017-11-17,80.00,call,35.83,67.18,65.15,4,69.20,5,0,0,2017-11-15,3.43164,0.99422,0.00044,-0.15627,0.00179,0.00433
4,IBM171117P00080000,IBM,2017-11-17,80.00,put,0.00,0.01,0.00,0,0.03,50,0,0,2017-11-15,2.78577,-0.00112,0.00012,-0.02827,0.00041,-0.00001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,IBM200117P00210000,IBM,2020-01-17,210.00,put,33.15,68.00,65.50,35,70.50,25,0,2,2017-11-15,0.30489,-0.69546,0.00530,-0.00918,0.75897,-3.70003
995,IBM200117C00220000,IBM,2020-01-17,220.00,call,1.55,1.53,1.06,47,2.00,19,0,1104,2017-11-15,0.17715,0.09434,0.00438,-0.00446,0.36459,0.26827
996,IBM200117P00220000,IBM,2020-01-17,220.00,put,37.15,77.00,74.50,28,79.50,20,0,9,2017-11-15,0.31540,-0.71876,0.00493,-0.00874,0.73144,-3.96990
997,IBM200117C00230000,IBM,2020-01-17,230.00,call,1.23,1.88,1.05,2,2.70,20,154,1085,2017-11-15,0.20153,0.10181,0.00407,-0.00532,0.38554,0.28457


In [10]:
exp_dates = ibm['expiration'].unique()

KeyError: 'expiration'